In [1]:
!pip3 install .

Processing c:\users\alex4321\documents\alpaca_lora_4bit
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached peft-0.3.0.dev0-py3-none-any.whl
  Cloning https://github.com/huggingface/transformers.git to c:\users\alex4321\appdata\local\temp\pip-install-i4zz5xjz\transformers_5ff4772c58d644099279b0cb9c4f2959
  Resolved https://github.com/huggingface/transformers.git to commit 1e9da2b0a6ef964c2cf72dd715dbee991a3f49fa
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for alpaca-lora-4bit: filename=alpaca_lora_4bit-0.1.2-cp310-cp310-win_amd64.whl size=221342 sha256=b1f2b4f5bc41470f1a3ce17b4e385a29406018e26525e39909e03e5a6f891d5f
  Stored 

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\alex4321\AppData\Local\Temp\pip-install-i4zz5xjz\transformers_5ff4772c58d644099279b0cb9c4f2959'


In [2]:
import torch

In [3]:
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, Autograd4bitQuantLinear, switch_backend_to

switch_backend_to("cuda")

Triton not found. Please run "pip install triton".

c:\Users\alex4321\anaconda3\envs\llama\lib\site-packages\tqdm-4.65.0-py3.10.egg\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Using CUDA implementation.


In [4]:
from alpaca_lora_4bit import autograd_4bit
from alpaca_lora_4bit import matmul_utils_4bit as mm4b

In [5]:
def _patched_linear_forward(self, x):
    assert not torch.isnan(x).any().item()
    assert not torch.isinf(x).any().item()
    if self.bits == 4:
        if torch.is_grad_enabled():
            out = autograd_4bit.AutogradMatmul4bit.apply(x, self.qweight, self.scales,
                                        self.qzeros if not self.is_v1_model else self.zeros,
                                        self.g_idx, self.bits, self.maxq)
            assert not torch.isnan(out).any().item()
            assert not torch.isinf(out).any().item()
        else:
            out = autograd_4bit.matmul4bit_with_backend(x, self.qweight, self.scales,
                                        self.qzeros if not self.is_v1_model else self.zeros,
                                        self.g_idx, self.bits, self.maxq, self.groupsize)
            assert not torch.isnan(out).any().item()
            assert not torch.isinf(out).any().item()
    elif self.bits == 2:
        raise NotImplementedError("Debugging 4-bit case")
        out = AutogradMatmul2bit.apply(x, self.qweight, self.scales, self.qzeros, self.g_idx, self.bits, self.maxq)
    else:
        raise NotImplementedError("Debugging 4-bit case")
        raise ValueError('Unsupported bitwidth.')
    if not self.disable_bias:
        assert not torch.isnan(self.bias).any().item()
        assert not torch.isinf(self.bias).any().item()
        out += self.bias
        assert not torch.isnan(out).any().item()
        assert not torch.isinf(out).any().item()
    return out


autograd_4bit.Autograd4bitQuantLinear.forward = _patched_linear_forward

In [6]:
def _patched_matmul4bit_with_backend(x, qweight, scales, qzeros, g_idx, bits, maxq, groupsize=None):
    assert not torch.isinf(x).any().item()
    assert not torch.isnan(x).any().item()
    out = mm4b.matmul4bit(x, qweight, scales, qzeros, g_idx, groupsize)
    assert not torch.isinf(out).any().item()
    assert not torch.isnan(out).any().item()
    return out


autograd_4bit.matmul4bit_with_backend = _patched_matmul4bit_with_backend

In [7]:
def build_forward_check_nan_inf(name, module):
    def _func(*args, **kwargs):
        input_tensors = [item for item in args if isinstance(item, torch.Tensor)] + \
            [item for item in kwargs.values() if isinstance(item, torch.Tensor)]
        for tensor in input_tensors:
            if torch.isinf(tensor).any().item():
                raise ValueError(f"Got Inf in {name} input")
            if torch.isnan(tensor).any().item():
                raise ValueError(f"Got NaN in {name} input")
        result = module._nan_check_old_forward(*args, **kwargs)
        tensors = []
        if isinstance(result, torch.Tensor):
            tensors = result
        if isinstance(result, tuple) or isinstance(result, list):
            tensors = [item for item in result if isinstance(item, torch.Tensor)]
        if isinstance(result, dict):
            tensors = [item for item in result.values() if isinstance(item, torch.Tensor)]
        for tensor in tensors:
            if torch.isinf(tensor).any().item():
                raise ValueError(f"Got Inf in {name} output")
            if torch.isnan(tensor).any().item():
                raise ValueError(f"Got NaN in {name} output")
        return result
    
    return _func

In [8]:
def wrap_forward_check_nan_inf(module, name):
    module._nan_check_old_forward = module.forward
    module.forward = build_forward_check_nan_inf(name, module)

In [9]:
model, tokenizer = load_llama_model_4bit_low_ram("vicuna-13B-1.1-GPTQ-4bit-128g",
                                                 "vicuna-13B-1.1-GPTQ-4bit-128g/vicuna-13B-1.1-GPTQ-4bit-128g.compat.no-act-order.pt",
                                                 groupsize=128)

Loading Model ...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loaded the model in 6.38 seconds.


In [10]:
wrap_forward_check_nan_inf(model.model.embed_tokens, "embed_tokens")
for i in range(40):
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn.q_proj, f"Layer {i} self-attention q_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn.k_proj, f"Layer {i} self-attention k_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn.v_proj, f"Layer {i} self-attention v_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn.o_proj, f"Layer {i} self-attention o_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn.rotary_emb, f"Layer {i} self-attention rotary_emb")
    wrap_forward_check_nan_inf(model.model.layers[i].self_attn, f"Layer {i} self-attention itself")
    wrap_forward_check_nan_inf(model.model.layers[i].mlp.gate_proj, f"Layer {i} mlp gate_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].mlp.down_proj, f"Layer {i} mlp down_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].mlp.up_proj, f"Layer {i} mlp up_proj")
    wrap_forward_check_nan_inf(model.model.layers[i].mlp.act_fn, f"Layer {i} mlp act_fn")
    wrap_forward_check_nan_inf(model.model.layers[i].mlp, f"Layer {i} mlp itself")
    wrap_forward_check_nan_inf(model.model.layers[i].input_layernorm, f"Layer {i} input_layernorm")
    wrap_forward_check_nan_inf(model.model.layers[i].post_attention_layernorm, f"Layer {i} post_attention_layernorm")
wrap_forward_check_nan_inf(model.model.norm, "norm")
wrap_forward_check_nan_inf(model.lm_head, "lm_head")

In [11]:
model.half()
for n, m in model.named_modules():
    if isinstance(m, Autograd4bitQuantLinear):
        if m.is_v1_model:
            m.zeros = m.zeros.half()
        m.scales = m.scales.half()
        m.bias = m.bias.half()
model.tie_weights()

In [12]:
from alpaca_lora_4bit.amp_wrapper import AMPWrapper
wrapper = AMPWrapper(model)
wrapper.apply_generate()

In [13]:
prompt = '''I think the meaning of life is'''
batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
batch = {k: v.cuda() for k, v in batch.items()}

In [14]:
with torch.no_grad():
     generated = model.generate(inputs=batch["input_ids"],
                                do_sample=True, use_cache=True,
                                repetition_penalty=1.1,
                                max_new_tokens=1,
                                temperature=0.5,
                                top_p=0.97,
                                top_k=40,
                                return_dict_in_generate=True,
                                output_attentions=False,
                                output_hidden_states=False,
                                output_scores=False)

AssertionError: 